In [1]:
import random
from mxnet import nd
from mxnet import autograd
import numpy as np
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

# Generating Data Sets 

In [2]:
num_inputs = 2
num_examples = 1000
true_w = nd.array([2, -3.4])
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = nd.dot(features, true_w) + true_b
labels += nd.random.normal(scale=0.1, shape=(labels.shape))

In [3]:
features[0], labels[0]

(
 [1.1630787 0.4838046]
 <NDArray 2 @cpu(0)>, 
 [4.8652573]
 <NDArray 1 @cpu(0)>)

In [4]:
scatter = go.Scatter3d(
    x=features[:, 0].asnumpy(),
    y=features[:, 1].asnumpy(),
    z=labels.asnumpy(),
    mode='markers',
    marker=dict(
        size=4,
        opacity=0.8
    )
)
fig = go.Figure(data=(scatter,))
py.offline.iplot(fig)

# Reading Data 

We read data in mini-batch per iteration. This has mostly to do with efficiency when optimizing. GPUs are much faster when it comes to dealing with matrices instead of loop through single vectors.

In [5]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = nd.array(indices[i:min(i + batch_size, num_examples)])
        yield features.take(j), labels.take(j)

In [6]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, y)
    break


[[-1.0092957   0.62100756]
 [-0.2215275  -0.6526793 ]
 [-0.14543211  0.22957022]
 [-0.65829736 -0.89476556]
 [-0.10911726  0.0257536 ]
 [-0.6047001   0.39736277]
 [ 0.26307964  0.86063385]
 [-1.2705711   0.6717436 ]
 [-0.30746168 -0.8484733 ]
 [ 1.9346759  -0.28565592]]
<NDArray 10x2 @cpu(0)> 
[ 0.0289273   5.926027    3.0967126   5.9641147   3.8003304   1.6289258
  1.8138908  -0.66895676  6.6559415   9.000041  ]
<NDArray 10 @cpu(0)>


# Initialize Model Parameters

Weights are initialized with normal random numbers, bias are set to all zeros.

In [7]:
w = nd.random.normal(scale=0.01, shape=(num_inputs, 1))
# w = nd.zeros(shape=(num_inputs, 1))
b = nd.zeros(shape=(1,))

In order for `autograd` to know that weights and bias need to be updated during training, we need call `attach_grad` on $\mathbf{w}$ and $b$.

In [8]:
w.attach_grad()
b.attach_grad()

# Define the Model

In [9]:
def linreg(X, w, b):
    return nd.dot(X, w) + b

# Define the Loss

In [10]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

# Define the Optimization Algorithm

The gradient calculated by automatic differentiation is the gradient sum of a batch of example, so we divide it by the batch size to get the average.

In [11]:
def sgd(params, lr, batch_size):
    for param in params:
        param[:] = param - lr * param.grad / batch_size

# Training 

In [12]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        with autograd.record():
            l = loss(net(X, w, b), y)
        l.backward()
        sgd([w, b], lr, batch_size)
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

epoch 1, loss 0.040771
epoch 2, loss 0.004901
epoch 3, loss 0.004848


In [13]:
print('Error in estimating w: ', true_w - w.reshape(true_w.shape))
print('Error in estimating b: ', true_b - b)

Error in estimating w:  
[ 0.00077772 -0.0044415 ]
<NDArray 2 @cpu(0)>
Error in estimating b:  
[-0.00444031]
<NDArray 1 @cpu(0)>


In [14]:
x = features[:, 0].asnumpy()
y = features[:, 1].asnumpy()
datapoint = go.Scatter3d(
    x=x,
    y=y,
    z=labels.asnumpy(),
    mode='markers',
    marker=dict(
        size=4,
        opacity=0.8
    )
)
true_surface = go.Mesh3d(
    x=x,
    y=y,
    z=x * true_w[0].asscalar() + y * true_w[1].asscalar() + true_b,
    opacity=0.4
)
estimated_surface = go.Mesh3d(
    x=x,
    y=y,
    z=x * w[0].asscalar() + y * w[1].asscalar() + b.asscalar(),
    opacity=0.4
)
fig = go.Figure(data=(scatter, true_surface, estimated_surface))
py.offline.iplot(fig)

# Problems 

1. What would happen if we were to initialize the weights $\mathbf{w} = 0$. Would the algorithm still work?

It still works. In multi-layer neural network, weights can't be initialized to all zero because of the symmetry problem which cause all layers always have the same weights. But in linear regression or logistic regression it will not cause any problem.

2. Assume that you're [Georg Simon Ohm](https://en.wikipedia.org/wiki/Georg_Ohm) trying to come up with a model between voltage and current. Can you use `autograd` to learn the parameters of your model.

3. Can you use [Planck's Law](https://en.wikipedia.org/wiki/Planck%27s_law) to determine the temperature of an object using spectral energy density.

4. What are the problems you might encounter if you wanted to extend `autograd` to second derivatives? How would you fix them?

5.  Why is the `reshape` function needed in the `squared_loss` function?

6. Experiment using different learning rates to find out how fast the loss function value drops.

7. If the number of examples cannot be divided by the batch size, what happens to the `data_iter` function's behavior?